<a href="https://colab.research.google.com/github/thxxx/datascience/blob/master/hospital5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import 모음
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
from sklearn.ensemble import RandomForestClassifier
import xgboost

import numpy as np
import graphviz

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/data/dacon/'
os.listdir(path) # path에 뭐가 들어있는지 보여준다.

['submission_sample.csv',
 'test.csv',
 'train.csv',
 'hospital.ipynb',
 'hospital2.ipynb',
 'Untitled',
 'hospital3.ipynb',
 'hospital3.ipynb의 사본']

In [ ]:
# 파일 전부 읽는다.
tdf = pd.read_csv(os.path.join(path + 'train.csv'), encoding='utf-8')
test_df = pd.read_csv(os.path.join(path + 'test.csv'), encoding='utf-8')
sdf = pd.read_csv(os.path.join(path + 'submission_sample.csv'), encoding='utf-8')

In [ ]:
tdf.shape

(301, 58)

##Train data의 전처리 작업

In [ ]:
# revenue1이 NaN인건 다른 칼럼도 다 NaN인 row길래 드랍해버림
# 훨씬 코드를 잘짤 수 있다. 일단은 생각이 안나서 이따위로 함
# for i, x in enumerate(list(tdf['revenue1'].isna())):
#   if x == True:
#     print(tdf.drop(i, axis=0, inplace=True))
tdf.dropna(subset=['revenue1'], inplace=True)

# inst_id 는 아이디일 뿐인데 평가에 사용되는 듯해서 드랍해버렸다
tdf = tdf.drop(['inst_id'], axis=1)
# ownerChange의 Nan값들은 same으로 채운다. same이 241, change가 42개, NaN이 10개
tdf['ownerChange'] = tdf['ownerChange'].fillna('same')
# 오픈했는지 아닌지 OC open, close를 숫자 0,1로 바꿔준다.
tdf['OC'] = tdf['OC'].apply(lambda x: 1 if x=='open' else 0)

# String same, change를 0,1로 바꾼다.
tdf['ownerChange'] = tdf['ownerChange'].apply(lambda x: 0 if x=='same' else 1 )
# bedCount, employee1,2 다 그냥 평균값으로 NaN 채움.
tdf['bedCount'] = tdf['bedCount'].fillna(tdf['bedCount'].mean()) # NaN인건 5개, 
tdf['employee1'] = tdf['employee1'].fillna(tdf['employee1'].mean())
tdf['employee2'] = tdf['employee2'].fillna(tdf['employee2'].mean())

In [ ]:
# 원핫 인코딩 아주 간단한 방법
tdf = pd.get_dummies(tdf, columns=['sido', 'instkind'])

In [ ]:
# 값들의 정규화. MinMaxScaler는 데이터 값을 0과 1사이 값으로 변환
scaler = MinMaxScaler(feature_range = (0,1))
# scaler.fit(tdf)
tdf[tdf.columns] = scaler.fit_transform(tdf[tdf.columns])
print("HI!")


HI!


In [ ]:
# OC는 정답이니까 따로 빼낸다!

skf = StratifiedKFold(n_splits=5)
n_iter = 0

train_label = tdf['OC']
tdf = tdf.drop(['OC'], axis=1)

In [ ]:
from sklearn.decomposition import PCA

# n_components는 pca로 반환할 차원의 수
pca = PCA(n_components=50)

#fit과 transform을 호출해 PCA변환 데이터 반환
pca.fit(tdf)
tdf = pca.transform(tdf)


In [ ]:

columns = []
for i in range(50):
  columns.append(f"pca_com{i+1}")
pca_train = pd.DataFrame(tdf, columns=columns)


In [ ]:
# 트레인 데이터 test, train으로 나눔
X_train, X_test, y_train, y_test = train_test_split(pca_train.values, train_label, test_size=0.3, random_state=177)

In [ ]:
print(xgboost.__version__)

0.90


In [ ]:
dtrain = xgboost.DMatrix(data=X_train, label=y_train)
dtest = xgboost.DMatrix(data=X_test, label=y_test)

In [ ]:
params = {
    'max_depth':3,
    'eta':0.1,
    'objective':'binary:logistic',
    'eval_metric':'logloss',
    'early_stoppings':100
}
num_rounds=400

In [ ]:
wlist = [(dtrain, 'train'),(dtest,'test')]
xgbmodel = xgboost.train(params=params, dtrain=dtrain, num_boost_round=num_rounds, early_stopping_rounds=100, evals=wlist)

[0]	train-logloss:0.616494	test-logloss:0.620079
Multiple eval metrics have been passed: 'test-logloss' will be used for early stopping.

Will train until test-logloss hasn't improved in 100 rounds.
[1]	train-logloss:0.551681	test-logloss:0.564622
[2]	train-logloss:0.497788	test-logloss:0.512051
[3]	train-logloss:0.451355	test-logloss:0.472223
[4]	train-logloss:0.412138	test-logloss:0.433945
[5]	train-logloss:0.3775	test-logloss:0.406473
[6]	train-logloss:0.347212	test-logloss:0.37846
[7]	train-logloss:0.321592	test-logloss:0.35298
[8]	train-logloss:0.298293	test-logloss:0.33188
[9]	train-logloss:0.277189	test-logloss:0.318104
[10]	train-logloss:0.259382	test-logloss:0.300846
[11]	train-logloss:0.242549	test-logloss:0.29078
[12]	train-logloss:0.228437	test-logloss:0.277332
[13]	train-logloss:0.214754	test-logloss:0.270439
[14]	train-logloss:0.200516	test-logloss:0.260474
[15]	train-logloss:0.187813	test-logloss:0.251565
[16]	train-logloss:0.176476	test-logloss:0.243728
[17]	train-loglo

In [ ]:
pred_probs = xgbmodel.predict(dtest)
print(np.round(pred_probs[:10],3))

[0.998 0.986 0.823 0.995 0.996 0.998 0.999 0.998 0.975 0.993]


In [ ]:
preds = [1 if x>0.5 else 0 for x in pred_probs]
len(preds)

88

In [ ]:
print(accuracy_score(y_test, preds))

0.9318181818181818


In [ ]:
from xgboost import XGBClassifier

xgb_wrapper = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
xgb_wrapper.fit(X_train, y_train)
w_preds = xgb_wrapper.predict(X_test)
w_pred_proba = xgb_wrapper.predict_proba(X_test)[:,1]

0.9318181818181818


In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100],
    'max_depth':[6,8,10,12],
    'min_samples_leaf':[8,12,18],
    'min_samples_split':[8,16,20]
}
# 랜덤포레스트분류기 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(X_train, y_train)

print(f"최적 하이퍼 파라미터 {grid_cv.best_params_}")
print(f"최고 예측 정확도 {grid_cv.best_score_}")

최적 하이퍼 파라미터 {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 100}
최고 예측 정확도 0.9463639824861984


In [ ]:
# 위에서 얻은 최적 하이퍼 파라미터 대로 실행
rf_clf2 = RandomForestClassifier(random_state=0, max_depth = 6, min_samples_leaf = 8, min_samples_split = 8, n_estimators = 100)

rf_clf2.fit(X_train, y_train)
# 예측
pred2 = rf_clf2.predict(X_test)

In [ ]:
# 정확도!
print("정확도 : ",accuracy_score(y_test, pred2))

정확도 :  0.9545454545454546


In [ ]:
pred2

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.])

In [ ]:
# 각 피처별 중요도를 그래프로 보여준다.
plt.figure(figsize=(25,18))

# sns.barplot(x=tdf.columns, y=dt_clf.feature_importances_)

feature = []
scoress = []


for index, score  in enumerate(rf_clf2.feature_importances_):
  if score > 0:
    print(tdf.columns[index], "  : ", score)
    feature.append(tdf.columns[index])
    scoress.append(score)

sns.barplot(x=scoress, y=feature)


AttributeError: ignored

<Figure size 1800x1296 with 0 Axes>

In [ ]:
#Removing the comma in the employee1 and 2 columns in the test dataset and replace it with empty space and convert it to float format.
test_df.employee1 = test_df.employee1.astype('str').str.replace(",", "").astype('float')
test_df.employee2 = test_df.employee2.astype('str').str.replace(",", "").astype('float')

In [ ]:
# inst_id 는 아이디일 뿐인데 평가에 사용되는 듯해서 드랍해버렸다
test_df = test_df.drop(['inst_id'], axis=1)
test_df = test_df.drop(['sido'], axis=1)
test_df = test_df.drop(['instkind'], axis=1)

# ownerChange의 Nan값들은 same으로 채운다. same이 241, change가 42개, NaN이 10개
test_df['ownerChange'] = test_df['ownerChange'].fillna('same')

# 오픈했는지 아닌지 OC open, close를 숫자 0,1로 바꿔준다.
test_df['OC'] = test_df['OC'].apply(lambda x: 1 if x=='open' else 0)

# String same, change를 0,1로 바꾼다.
test_df['ownerChange'] = test_df['ownerChange'].apply(lambda x: 0 if x=='same' else 1 )

# 원핫 인코딩 아주 간단한 방법
# test_df = pd.get_dummies(test_df, columns=['sido', 'instkind'])

# bedCount, employee1,2 다 그냥 평균값으로 NaN 채움.
test_df['bedCount'] = test_df['bedCount'].fillna(test_df['bedCount'].mean()) # NaN인건 5개, 
test_df['employee1'] = test_df['employee1'].fillna(test_df['employee1'].mean())
test_df['employee2'] = test_df['employee2'].fillna(test_df['employee1'].mean())
test_df['openDate'] = test_df['openDate'].fillna(test_df['openDate'].mean())

# 값들의 정규화. MinMaxScaler는 데이터 값을 0과 1사이 값으로 변환
scaler = MinMaxScaler(feature_range = (0,1))
# scaler.fit(test_df)
test_df[test_df.columns] = scaler.fit_transform(test_df[test_df.columns])
print("HI!")

train_label = test_df['OC']
test_df = test_df.drop(['OC'], axis=1)

In [ ]:

#     print(tdf.drop(i, axis=0, inplace=True))
test_df.dropna(subset=['revenue1'], inplace=True)
test_df['openDate'] = test_df['openDate'].fillna(test_df['openDate'].mean())

In [ ]:
pred3 = rf_clf2.predict(test_df.values)

In [ ]:
pred3